## Introduction to tensors


This notebook makes a short introduction to working with *tensors*, the fundamental object of Deep Learning frameworks, such as PyTorch or Tensorflow.

Generally speaking, tensors are N-dimensional arrays, and can be considered an extension of classical NumPy arrays. The core difference between the two, is that tensors are specifically designed to be run on GPUs, making tensor operations a few orders of magnitude faster than their CPU counter-parts.

In [ ]:
!pip install ipython-autotime
import torch
import numpy as np

# We will use the autotime command to get the running time of each code block and investigate the difference
%load_ext autotime

First, lets create two similar N-D arrays: an np array and a tensor, of the same dimensions. We will populate them with:
- [`np.random.rand()`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.rand.html)
- [`torch.rand()`](https://pytorch.org/docs/stable/generated/torch.rand.html)


In [ ]:
array_size = (3, 3)

# 2D arrays
array1 = np.random.rand(*array_size)
array2 = np.random.rand(*array_size)

# 2D tensors
tensor1 = torch.rand(*array_size)
tensor2 = torch.rand(*array_size)

Let's print the data. We can also have a look at the type of data (`.dtype` attribute)

In [ ]:
print('Np array values:')
print(array1)
print('\nTensor values:')
print(tensor1)

print(f'\nNp array data type is {array1.dtype}')
print(f'Tensor data type is {tensor1.dtype}')

Tensors, unlike np arrays, can be run on both CPU and GPU. They have an attribute, `.device` that holds this particular information. Let's print and see where the tensor resides upon default creation.

In [ ]:
print(tensor1.device)

Now let's see how fast the operations are on both types of arrays.

First, run a multiplication operation on the np arrays with [`np.matmul()`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html#numpy-matmul):

In [31]:
res_np = np.matmul(array1, array2)

time: 875 ms (started: 2025-03-25 00:28:06 +02:00)


Then, a similar multiplication on the tensors with [`torch.matmul()`](https://pytorch.org/docs/stable/generated/torch.matmul.html#torch.matmul):

In [30]:
res_tensor = torch.matmul(tensor1, tensor2)

time: 0 ns (started: 2025-03-25 00:28:04 +02:00)


The difference in running time is mainly due to the different types of data contained by the np arrays and the tensors. By default, np arrays use `float64`, whereas tensors use `float32` data. We can convert the np arrays to a similar data type and re-run the same operations. We can use the `np.float32()` cast operator or the [`np.ndarray.astype()`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.astype.html) function.

In [ ]:
array1_float32 = array1.astype(np.float32)
array2_float32 = array2.astype(np.float32)

In [ ]:
res_np_float32 = np.matmul(array1_float32, array2_float32)

In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

Now the two operations run in a similar time span. So let's see the GPU speedup (if any). In order to do so, we must first check if our system has a dedicated GPU with CUDA support. We can check with `torch.cuda.is_available()`.

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
print(device)

If there is such a device available, then we can load the tensors on it, with the [`torch.Tensor.to()`](https://pytorch.org/docs/stable/generated/torch.Tensor.to.html) method.

In [ ]:
tensor1 = tensor1.to(device)
tensor2 = tensor2.to(device)

And then we re-run the multiplication, only that this time it will be done on the GPU, instead of the CPU.

In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

Well, the difference is not spectacular at all, but that is due to the very small size of the arrays. We have to pump those numbers up! Those are rookie numbers.

So we will re-run the above operations, but on a larger scale. Switch `array_size` to 100 x 100 x 100 x 100 and see what the GPU does now.

In [ ]:
array_size = (100, 100, 100, 100)
array1 = np.float32(np.random.rand(*array_size))
array2 = np.float32(np.random.rand(*array_size))
tensor1 = torch.rand(*array_size).to(device)
tensor2 = torch.rand(*array_size).to(device)

In [ ]:
res_np = np.matmul(array1, array2)

In [ ]:
res_tensor = torch.matmul(tensor1, tensor2)

In order to extract numeric values from the tensor and use them as Python numbers, we have several options:
- index the tensor and then call [`torch.Tensor.item()`](https://pytorch.org/docs/stable/generated/torch.Tensor.item.html) on a single value;
- convert the tensor to a list [`torch.Tensor.tolist()`](https://pytorch.org/docs/stable/generated/torch.Tensor.tolist.html#torch.Tensor.tolist) and treat it as a Python list;
- convert the tensor to a np array [`torch.Tensor.numpy()`]() and treat it as a np array - this step needs the tensor to be transfered back on CPU before-hand with: `torch.Tensor.cpu()`.

In [ ]:
x = res_tensor.cpu().numpy()
print(x[0][0])